Vanillin production
------------------

In 2010, Brochado *et al* used heuristic optimization together with flux simulations to design a vanillin producing yeast strain.

Brochado, A. R., Andrejev, S., Maranas, C. D., & Patil, K. R. (2012). Impact of stoichiometry representation on simulation of genotype-phenotype relationships in metabolic networks. PLoS Computational Biology, 8(11), e1002758. doi:10.1371/journal.pcbi.1002758

Genome-scale metabolic model
--------------------------

In their work, the authors used *iFF708* model, but recent insights in Yeast yielded newer and more complete versions. 
Becuase this algorithms should be agnostic to the model, we implement the same strategy with a newer model.

In [1]:
from cameo import models

In [2]:
model = models.bigg.iMM904

Constraints can be set in the model according to data found in the literature. The defined conditions allow the simulation of phenotypes very close to the experimental results.

<img src=http://www.biomedcentral.com/content/figures/1752-0509-7-36-2.jpg/>
**Model validation by comparing in silico prediction of the specific growth rate with experimental data**. Growth phenotypes were collected from literature and compared to simulated values for chemostat cultivations at four different conditions, nitrogen limited aerobic (green) and anaerobic (red), carbon limited aerobic (blue) and anaerobic (white).


Österlund, T., Nookaew, I., Bordel, S., & Nielsen, J. (2013). Mapping condition-dependent regulation of metabolism in yeast through genome-scale modeling. BMC Systems Biology, 7, 36. doi:10.1186/1752-0509-7-36

In [3]:
model.reactions.EX_glc__D_e.lower_bound = -13 #glucose exchange
model.reactions.EX_o2_e.lower_bound = -3  #oxygen exchange

In [4]:
model.medium

,reaction_id,reaction_name,lower_bound,upper_bound
0,EX_fe2_e,Fe2+ exchange,-999999.0,999999.0
1,EX_glc__D_e,D-Glucose exchange,-13.0,999999.0
2,EX_h2o_e,H2O exchange,-999999.0,999999.0
3,EX_h_e,H+ exchange,-999999.0,999999.0
4,EX_k_e,K+ exchange,-999999.0,999999.0
5,EX_na1_e,Sodium exchange,-999999.0,999999.0
6,EX_nh4_e,Ammonia exchange,-999999.0,999999.0
7,EX_o2_e,O2 exchange,-3.0,999999.0
8,EX_pi_e,Phosphate exchange,-999999.0,999999.0
9,EX_so4_e,Sulfate exchange,-999999.0,999999.0


In [5]:
model.objective = model.reactions.BIOMASS_SC5_notrace #growth
model.optimize().f

0.39022235350798895

Heterologous pathway
-------------------

Vanillin is not produced by *S. cervisiae*. In their work an heterolgous pathway is inserted to allow generate a vanillin production strain. The pathway is described as:

<img src=http://static-content.springer.com/image/art%3A10.1186%2F1475-2859-9-84/MediaObjects/12934_2010_Article_474_Fig1_HTML.jpg>
**Schematic representation of the de novo VG biosynthetic pathway in S. Cerevisisae** (as designed by Hansen et al [5]). Metabolites are shown in black, enzymes are shown in black and in italic, cofactors and additional precursors are shown in red. Reactions catalyzed by heterologously introduced enzymes are shown in red. Reactions converting glucose to aromatic amino acids are represented by dashed black arrows. Metabolite secretion is represented by solid black arrows where relative thickness corresponds to relative extracellular accumulation. 3-DSH stands for 3-dedhydroshikimate, PAC stands for protocathechuic acid, PAL stands for protocatechuic aldehyde, SAM stands for S-adenosylmethionine. 3DSD stands for 3-dedhydroshikimate dehydratase, ACAR stands for aryl carboxylic acid reductase, PPTase stands for phosphopantetheine transferase, hsOMT stands for O-methyltransferase, and UGT stands for UDP-glycosyltransferase. Adapted from Hansen et al. [5].
Brochado et al. Microbial Cell Factories 2010 9:84   doi:10.1186/1475-2859-9-84

Using **cameo**, is very easy to generate a pathway and add it to a model.

In [6]:
from cameo.strain_design.pathway_prediction import PathwayPredictor

In [7]:
predictor = PathwayPredictor(model)

In [12]:
pathways = predictor.run('vanillin', max_predictions=3)

,equation,lower_bound,upper_bound
MNXR5336,H2O + vanillin + NAD(+) <=> 2.0 H(+) + NADH(2-...,-1000,1000
MNXR5340,"3,4-dihydroxybenzoate + NAD(+) + H2O + formald...",-1000,1000
MNXR14769,"H2O + 3,4-dihydroxybenzoate + NAD(+) <=> NADH(...",-1000,1000


,equation,lower_bound,upper_bound
MNXR5336,H2O + vanillin + NAD(+) <=> 2.0 H(+) + NADH(2-...,-1000,1000
MNXR5340,"3,4-dihydroxybenzoate + NAD(+) + H2O + formald...",-1000,1000
MNXR68718,"H2O + 3,4-dihydroxybenzoate <=> 3-dehydroshiki...",-1000,1000


,equation,lower_bound,upper_bound
MNXR230,"3,4-dihydroxybenzoate + NADP(+) + H2O <=> O2 +...",-1000,1000
MNXR5336,H2O + vanillin + NAD(+) <=> 2.0 H(+) + NADH(2-...,-1000,1000
MNXR5340,"3,4-dihydroxybenzoate + NAD(+) + H2O + formald...",-1000,1000


In [13]:
vanillin_pathway = pathways.pathways[0]

In [6]:
from cameo.core.pathway import Pathway

In [7]:
vanillin_pathway = Pathway.from_file("data/vanillin_pathway.tsv")
vanillin_pathway.data_frame

,equation,lower_bound,upper_bound
3DSD,3-dehydroshikimate --> H2O + protocathechuic acid,0.0,1000.0
ACAR_PPTase,ATP + protocathechuic acid + NADPH --> ADP + p...,0.0,1000.0
hsOMT,S-adenosyl-L-methionine + protocatechuic aldeh...,0.0,1000.0
UGT,Vanillin + UDP-glucose --> vanillin-B-glucoside,0.0,1000.0


And now we can plug the pathway to the model.

In [14]:
vanillin_pathway.plug_model(model)

In [15]:
from cameo import phenotypic_phase_plane

The Phenotypic phase plane can be used to analyse the theoretical yields at different growth rates.

In [ ]:
production_envelope = phenotypic_phase_plane(model, variables=[model.reactions.BIOMASS_SC5_notrace], 
                                             objective=model.reactions.EX_vnl_b_glu_c)
production_envelope.plot()

In [16]:
production_envelope = phenotypic_phase_plane(model, variables=[model.reactions.BIOMASS_SC5_notrace], 
                                             objective=model.reactions.EX_vnl_b_glu_c)
production_envelope.plot()

AttributeError: DictList has no attribute or entry EX_vnl_b_glu_c

To find gene knockout targets, we use `cameo.strain_design.heuristic` package which implements the OptGene strategy.

The authors used the biomass-product coupled yield (bpcy) for optimization which is the equivalent of running OptGene in non-robust mode. All simulations were computed using MOMA but because **cameo** does not implement MOMA we use it's equivalent linear version (it minimizes the absolute distance instead of the quadratic distance). The linear MOMA version is faster than the original MOMA formulation.

By default, our OptGene implementation will run 20'000 evaluations.

In [11]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene
from cameo.flux_analysis.simulation import lmoma

In [12]:
optgene = OptGene(model)

In [13]:
results = optgene.run(target="EX_vnl_b_glu_c", 
                      biomass="BIOMASS_SC5_notrace",
                      substrate="EX_glc__D_e",
                      simulation_method=lmoma)

Starting optimization at Thu, 26 May 2016 13:53:54


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
/Users/joao/.virtualenvs/cameo06-py3/lib/python3.4/site-packages/ipywidgets/widgets/widget_string.py:55: UserWarning:

The Latex widget is deprecated. Use Label instead



Finished after 01:28:45


In [14]:
results

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
/Users/joao/.virtualenvs/cameo06-py3/lib/python3.4/site-packages/ipywidgets/widgets/widget_string.py:55: UserWarning:

The Latex widget is deprecated. Use Label instead



,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness
0,"(OHPBAT, G6PI, PSERT, GLCGSD, OMCDCm, PGI, VAL...","((YBR196C, YBL011W, YOR184W, YHR208W, YJL121C,...",8.0,0.0,0.009158,0.023639,0.176487,0.001896,0.000335
1,"(OHPBAT, GLCGSD, G6PI, PSERT, OMCDCm, PGI, VAL...","((YAR035W, YBR196C, YOR184W, YHR208W, YJL121C,...",8.0,0.0,0.009158,0.023639,0.176487,0.001896,0.000335
2,"(OHPBAT, G6PI, PSERT, GLCGSD, PGI, G6PI3, RPE,...","((YBR196C, YBL011W, YOR184W, YLR432W, YJL121C,...",8.0,0.0,0.009158,0.023565,0.177159,0.001890,0.000335
3,"(OHPBAT, PMI12346PH, GLCGSD, G6PI, PSERT, PGI,...","((YBR196C, YOR184W, YJL121C, YOR163W, YPR184W,...",8.0,0.0,0.009158,0.023565,0.177159,0.001890,0.000335
4,"(OHPBAT, G6PI, PSERT, ALATA_L, PGI, G6PI3, RPE...","((YDR111C, YBR196C, YOR184W, YOR377W, YJL121C,...",8.0,0.0,0.009063,0.023205,0.180413,0.001862,0.000336
5,"(OHPBAT, GLCGSD, G6PI, PSERT, PGI, AMETt2, G6P...","((YBR196C, YOR184W, YOR071C, YJL121C, YPR184W,...",8.0,0.0,0.009158,0.023565,0.177159,0.001890,0.000335
6,"(OHPBAT, FAS120ACPm, PSERT, PGI, FAS80ACPm_L, ...","((YAR035W, YBR196C, YER061C, YOR184W, YJL121C,...",8.0,0.0,0.009158,0.023565,0.177159,0.001890,0.000335
...,...,...,...,...,...,...,...,...,...
30,"(OHPBAT, ALLTTti, GLCGSD, G6PI, PSERT, PGI, G6...","((YAR035W, YBR196C, YOR184W, YJL121C, YPR184W,...",9.0,0.0,0.009158,0.023565,0.177159,0.001890,0.000335
31,"(OHPBAT, ALLTTti, GLCGSD, G6PI, PSERT, PGI, G6...","((YBR196C, YOR184W, YBR166C, YJL121C, YPR184W,...",9.0,0.0,0.009158,0.023494,0.177800,0.001891,0.000336
